In [92]:
from dotenv import load_dotenv
import urllib.parse as urlparse
import requests
import json
import os
from datetime import datetime

class Untappd:

    def __init__(self, clientId = "", clientSecret = ""):
        load_dotenv(dotenv_path="../.env")
        self._clientId = os.getenv("CLIENT_ID")
        self._clientSecret = os.getenv("CLIENT_SECRET")
        self._baseUrl = "https://api.untappd.com/v4/"


    def keys(self):
        return {
            "client_id": self._clientId,
            "client_secret": self._clientSecret
        }

    def findBrewery(self, brewery="", writeFile=False):
        methodUrl = self._baseUrl + "search/brewery"
        params = {
            "q": brewery
        }
        requestUrl = methodUrl + "?" + urlparse.urlencode({**params, **self.keys()})
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        breweries = [] 
        
        if response.status_code == 200:
            content = response.json()
            
            
            if content["response"]["found"] > 0:
                breweries = content["response"]["brewery"]["items"]
        
        return breweries, counter, response
    
    def breweryInfo(self, breweryId="", writeFile=False):
        methodUrl = self._baseUrl + "brewery/info/"
        requestUrl = methodUrl + str(breweryId) + "?" + urlparse.urlencode(self.keys())
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        
        if response.status_code == 200:
            content = response.json()

        return content["response"], counter, response



### Create a brewery dataset

In [178]:
import numpy as np
import pandas as pd
import pickle

try:
    with open ("../data/all_breweries.pkl", "rb") as fp:
        all_breweries = pickle.load(fp)
except:
    print( "No file existed, creating one!" )
    breweries = pd.read_csv("../data/Texas Breweries.csv")
    brewpubs = pd.read_csv("../data/Texas Brewpubs.csv")
    brewery_list = list(breweries.T.to_dict().values())
    all_breweries = brewery_list + list( brewpubs.T.to_dict().values() )

    #remove duplicates
    brewery_count = pd.DataFrame(all_breweries).groupby(["Name"])["Status"].count()
    dups = brewery_count.loc[ brewery_count > 1].reset_index()
    dups = dups["Name"].to_list()
    print("Duplicates to be removed", dups)
    brewery_parsed = [ brewery for brewery in all_breweries if brewery["Name"] not in dups ]

    #Make name searchable
    for brewery in all_breweries:
        brewery["Name"] = brewery["Name"].replace("Company","")
        brewery["Name"] = brewery["Name"].replace(".","")
        brewery["Name"] = brewery["Name"].replace("&"," ")
        brewery["Name"] = brewery["Name"].replace("Cooperative","")
    
    with open('../data/all_breweries.pkl', 'wb') as fp:
        pickle.dump(all_breweries, fp)
        
  
display( all_breweries[0:2] )

[{'Name': '111 Brewing',
  'Street Address': 111,
  'Road': 'S FANNIN AVENUE',
  'City': 'DENISON',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '4/10/2019',
  'Status': 'In the works',
  'untappd': [{'brewery': {'brewery_id': 66888,
     'beer_count': 65,
     'brewery_name': 'Milepost 111 Brewing Company',
     'brewery_slug': 'milepost-111-brewing-company',
     'brewery_page_url': '/milepostbeer',
     'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg',
     'country_name': 'United States',
     'location': {'brewery_city': 'Cashmere',
      'brewery_state': 'WA',
      'lat': 47.5244,
      'lng': -120.47}}}]},
 {'Name': '2 Docs Brewing Co',
  'Street Address': 502,
  'Road': 'TEXAS AVENUE',
  'City': 'LUBBOCK',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '10/5/2018',
  'Status': 'Open',
  'untappd': [{'brewery': {'brewery_id': 420745,
     'beer_count': 22,
     'brewery_name': 'Two

Due to untappd's 100 requests an hour, i'll need to run this over a couple of hours.  Thus saving results for future reference.

In [135]:
import pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

untappd = Untappd()
 
for brewery in all_breweries:
    if "untappd" not in brewery:
        brewery_result, counter, response = untappd.findBrewery(brewery["Name"])
        
        if len( brewery_result ) > 0:
            brewery["untappd"] = brewery_result
        else:
            brewery["untappd"] = [{"no_results":True}]

        clear_output(wait=True)
        display(counter)
        if counter == 0:
            break
        
#         break
#Save results
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)

print( "sample output" )
print( all_breweries[0] )

38

sample output
{'Name': '111 Brewing', 'Street Address': 111, 'Road': 'S FANNIN AVENUE', 'City': 'DENISON', 'State': 'TX', 'Brewery License': 'Brewery', 'License Date': '4/10/2019', 'Status': 'In the works', 'untappd': [{'brewery': {'brewery_id': 66888, 'beer_count': 65, 'brewery_name': 'Milepost 111 Brewing Company', 'brewery_slug': 'milepost-111-brewing-company', 'brewery_page_url': '/milepostbeer', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Cashmere', 'brewery_state': 'WA', 'lat': 47.5244, 'lng': -120.47}}}]}


In [136]:
import pickle
import pandas as pd
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)
 
# Normalized the results
brewery_parsed = []
i = 0
for brewery in all_breweries:
    clear_output(wait=True)
    display(i)
    i+=1
    for entries in brewery["untappd"]:
        if "no_results" not in entries:
            brewery_parsed.append({
                "Name":brewery["Name"],
                "Street Address":brewery["Street Address"],
                "Road":brewery["Road"],
                "City":brewery["City"],
                "State":brewery["State"],
                "Brewery License":brewery["Brewery License"],
                "License Date":brewery["License Date"],
                "Status":brewery["Status"],
                "search_count":len(brewery["untappd"]),
                "brewery_id":entries["brewery"]["brewery_id"],
                "brewery_name":entries["brewery"]["brewery_name"],
                "beer_count":entries["brewery"]["beer_count"],
                "brewery_city":entries["brewery"]["location"]["brewery_city"],
                "brewery_state":entries["brewery"]["location"]["brewery_state"],
                "brewery_lat":entries["brewery"]["location"]["lat"],
                "brewery_lng":entries["brewery"]["location"]["lng"]
            })
        

# Remove results where not in texas
brewery_parsed = [ brewery for brewery in brewery_parsed if "TX" in brewery["brewery_state"]]
    
# Must have 10 beers!
# brewery_parsed = [ brewery for brewery in brewery_parsed if brewery["beer_count"] > 9 ]

#remove duplicates
# brewery_count = pd.DataFrame(brewery_parsed).groupby(["Name"])["Status"].count()
# dups = brewery_count.loc[ brewery_count > 1].reset_index()
# dups = dups["Name"].to_list()
# print("Duplicates to be removed", dups)
# brewery_parsed = [ brewery for brewery in brewery_parsed if brewery["Name"] not in dups ]

# #Add Brewery ID back in to primary data
for brewery in all_breweries:
    for b in brewery_parsed:
        if brewery["Name"] in b["Name"]:
            brewery["brewery_id"] = b["brewery_id"]
            
# #Save results
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)
# #Convert to Pandas and save it
# # brewery_parsed = pd.DataFrame( brewery_parsed )
# # brewery_parsed.to_csv("../data/brewery_search.csv")
pd.options.display.max_rows = 999
display( pd.DataFrame( brewery_parsed ) )
pd.options.display.max_rows = 15

365

,Name,Street Address,Road,City,State,Brewery License,License Date,Status,search_count,brewery_id,brewery_name,beer_count,brewery_city,brewery_state,brewery_lat,brewery_lng
0,2 Docs Brewing Co,502,TEXAS AVENUE,LUBBOCK,TX,Brewery,10/5/2018,Open,1,420745,Two Docs Brewing Co.,22,Lubbock,TX,33.5909,-101.8460
1,3 Nations Brewing,2405,SQUIRE PLACE SUITE 200,FARMERS BRANCH,TX,Brewery,5/6/2015,Open,1,205645,3 Nations Brewing,43,Farmers Branch,TX,32.9310,-96.8983
2,4th Tap Brewing,10615,METRIC BOULEVARD,AUSTIN,TX,Brewery,4/13/2015,Open,1,229660,4th Tap Brewing Co-Op,90,Austin,TX,30.3852,-97.7119
3,512 Brewing,407,RADAM LANE 'F200',AUSTIN,TX,Brewery,5/20/2008,Open,1,1,(512) Brewing Company,156,Austin,TX,30.2236,-97.7697
4,8th Wonder Brewery,2202,DALLAS STREET,HOUSTON,TX,Brewery,12/29/2011,Open,1,45113,8th Wonder Brewery,157,Houston,TX,29.7492,-95.3558
5,Alamo Beer,415,BURNET STREET,SAN ANTONIO,TX,Brewery,9/29/2014,Open,1,43137,Alamo Beer Company,69,San Antonio,TX,29.4295,-98.4773
6,Altmeyer Lewis Brewing,15898,HWY 123 NORTH,SAN MARCOS,TX,Brewery,7/16/2015,Open,1,294420,Altmeyer & Lewis Brewing Company,0,San Marcos,TX,29.7874,-97.9467
7,Armadillo Ale Works,221,S BELL AVENUE,DENTON,TX,Brewery,2/19/2015,Open,1,9316,Armadillo Ale Works™,28,Denton,TX,33.2126,-97.1281
8,Athens Brewing,101,E TYLER STREET,ATHENS,TX,Brewery,4/18/2016,Open,1,267312,Athens Brewing Company,35,Athens,TX,32.2054,-95.8538
9,Austin Beerworks,3009,INDUSTRIAL TERRACE SUITE 150,AUSTIN,TX,Brewery,12/30/2010,Open,1,11819,Austin Beerworks,266,Austin,TX,30.3798,-97.7302


In [137]:
import pickle
import numpy as np
import pandas as pd
import math
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

untappd = Untappd()

i = 0
for brewery in all_breweries:
    if "brewery_id" in brewery and "brewery_info" not in brewery:
        brewery_info, counter, response = untappd.breweryInfo(brewery["brewery_id"])

        brewery["brewery_info"] = brewery_info
        
        clear_output(wait=True)
        display(counter)
        if counter == 0:
            break
        
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)
    
print("Sample output")
display(all_breweries[1]["brewery_info"])

print("Completed on ", datetime.now())


33

Sample output


{'brewery': {'brewery_id': 420745,
  'brewery_name': 'Two Docs Brewing Co.',
  'brewery_slug': 'two-docs-brewing',
  'brewery_page_url': '/TwoDocsBrewingCo',
  'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg',
  'brewery_label_hd': '',
  'country_name': 'United States',
  'brewery_in_production': 0,
  'is_independent': 1,
  'claimed_status': {'is_claimed': True,
   'claimed_slug': 'TwoDocsBrewingCo',
   'follow_status': False,
   'follower_count': 72,
   'uid': 6376655,
   'mute_status': 'none'},
  'beer_count': 22,
  'contact': {'twitter': '',
   'facebook': 'https://www.facebook.com/twodocsbrewing',
   'instagram': 'twodocsbrewing',
   'url': 'https://twodocsbrewing.com/'},
  'brewery_type': 'Micro Brewery',
  'brewery_type_id': 2,
  'location': {'brewery_address': '502 Texas Ave',
   'brewery_city': 'Lubbock',
   'brewery_state': 'TX',
   'brewery_lat': 33.5909,
   'brewery_lng': -101.846},
  'rating': {'count': 1943, 'rating_score': 3.79

Completed on  2019-11-09 17:50:29.236932


Flatten data

In [138]:
import pickle
import numpy as np
import pandas as pd
import math
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

for brewery in all_breweries:
    if "brewery_id" in brewery:
        brewery["brewery_name"] = brewery["brewery_info"]["brewery"]["brewery_name"]
        brewery["brewery_address"] = brewery["brewery_info"]["brewery"]["location"]["brewery_address"]
        brewery["brewery_city"] = brewery["brewery_info"]["brewery"]["location"]["brewery_city"]
        brewery["brewery_state"] = brewery["brewery_info"]["brewery"]["location"]["brewery_state"]
        brewery["brewery_lat"] = brewery["brewery_info"]["brewery"]["location"]["brewery_lat"]
        brewery["brewery_lng"] = brewery["brewery_info"]["brewery"]["location"]["brewery_lng"]
        brewery["beer_count"] = brewery["brewery_info"]["brewery"]["beer_count"]
        brewery["rating_count"] = brewery["brewery_info"]["brewery"]["rating"]["count"]
        brewery["rating_score"] = brewery["brewery_info"]["brewery"]["rating"]["rating_score"]
        brewery["checkins_total_count"] = brewery["brewery_info"]["brewery"]["stats"]["total_count"]
        brewery["checkins_unique_count"] = brewery["brewery_info"]["brewery"]["stats"]["unique_count"]
        brewery["checkins_monthly_count"] = brewery["brewery_info"]["brewery"]["stats"]["monthly_count"]
        brewery["checkins_weekly_count"] = brewery["brewery_info"]["brewery"]["stats"]["weekly_count"]
        brewery["age_on_service"] = brewery["brewery_info"]["brewery"]["stats"]["age_on_service"]
        brewery["brewery_in_production"] = brewery["brewery_info"]["brewery"]["brewery_in_production"]
        
        
pd.set_option('max_columns', 100)
display(pd.DataFrame(all_breweries).head())
pd.reset_option('max_columns')
                
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)

,Name,Street Address,Road,City,State,Brewery License,License Date,Status,untappd,brewery_id,brewery_info,brewery_name,brewery_address,brewery_city,brewery_state,brewery_lat,brewery_lng,beer_count,rating_count,rating_score,chekcins_total_count,chekcins_unique_count,chekcins_monthly_count,chekcins_weekly_count,age_on_service,brewery_in_production
0,111 Brewing,111,S FANNIN AVENUE,DENISON,TX,Brewery,4/10/2019,In the works,"[{'brewery': {'brewery_id': 66888, 'beer_count': 65, 'brewery_name': 'Milepost 111 Brewing Company', 'brewery_slug': 'milepost-111-brewing-company', 'brewery_page_url': '/milepostbeer', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Cashmere', 'brewery_state': 'WA', 'lat': 47.5244, 'lng': -120.47}}}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2 Docs Brewing Co,502,TEXAS AVENUE,LUBBOCK,TX,Brewery,10/5/2018,Open,"[{'brewery': {'brewery_id': 420745, 'beer_count': 22, 'brewery_name': 'Two Docs Brewing Co.', 'brewery_slug': 'two-docs-brewing-co', 'brewery_page_url': '/TwoDocsBrewingCo', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Lubbock', 'brewery_state': 'TX', 'lat': 33.5909, 'lng': -101.846}}}]",420745.0,"{'brewery': {'brewery_id': 420745, 'brewery_name': 'Two Docs Brewing Co.', 'brewery_slug': 'two-docs-brewing', 'brewery_page_url': '/TwoDocsBrewingCo', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg', 'brewery_label_hd': '', 'country_name': 'United States', 'brewery_in_production': 0, 'is_independent': 1, 'claimed_status': {'is_claimed': True, 'claimed_slug': 'TwoDocsBrewingCo', 'follow_status': False, 'follower_count': 72, 'uid': 6376655, 'mute_status': 'none'}, 'beer_count': 22, 'contact': {'twitter': '', 'facebook': 'https://www.facebook.com/twodocsbrewing', 'instagram': 'twodocsbrewing', 'url': 'https://twodocsbrewing.com/'}, 'brewery_type': 'Micro Brewery', 'brewery_type_id': 2, 'location': {'brewery_address': '502 Texas Ave', 'brewery_city': 'Lubbock', 'brewery_state': 'TX', 'brewery_lat': 33.5909, 'brewery_lng': -101.846}, 'rating': {'count': 1943, 'rating_score': 3.791}, 'brewery_description': 'We want to share our love of craft...",Two Docs Brewing Co.,502 Texas Ave,Lubbock,TX,33.5909,-101.8460,22.0,1943.0,3.791,2400.0,711.0,153.0,0.0,271.909155,0.0
2,3 Nations Brewing,2405,SQUIRE PLACE SUITE 200,FARMERS BRANCH,TX,Brewery,5/6/2015,Open,"[{'brewery': {'brewery_id': 205645, 'beer_count': 43, 'brewery_name': '3 Nations Brewing', 'brewery_slug': '3-nations-brewing', 'brewery_page_url': '/3NationsBrewing', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-205645_ed87a.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Farmers Branch', 'brewery_state': 'TX', 'lat': 32.931, 'lng': -96.8983}}}]",205645.0,"{'brewery': {'brewery_id': 205645, 'brewery_name': '3 Nations Brewing', 'brewery_slug': '3-nations-brewing', 'brewery_page_url': '/3NationsBrewing', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-205645_ed87a.jpeg', 'brewery_label_hd': '', 'country_name': 'United States', 'brewery_in_production': 0, 'is_independent': 1, 'claimed_status': {'is_claimed': True, 'claimed_slug': '3NationsBrewing', 'follow_status': False, 'follower_count': 1536, 'uid': 2671573, 'mute_status': 'none'}, 'beer_count': 44, 'contact': {'twitter': '3NationsBrewing', 'facebook': 'https://www.facebook.com/threenationsbrewing', 'instagram': '3nationsbrewing', 'url': 'https://3nationsbrewing.com/'}, 'brewery_type': 'Micro Brewery', 'brewery_type_id': 2, 'location': {'brewery_address': '2405 Squire Place, Suite 200', 'brewery_city': 'Farmers Branch', 'brewery_state': 'TX', 'brewery_lat': 32.931, 'brewery_lng': -96.8983}, 'rating': {'count': 67380, 'rating_score': 3.782

Let's research missing!

In [139]:
import pandas as pd
import pickle

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

df = pd.DataFrame(all_breweries)

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 200)
idx_filter = ( pd.isnull(df.brewery_id) ) & ( df.Status == "Open" )
print("Total need to research", sum( idx_filter))
display(df.loc[idx_filter,["Name","License Date","Brewery License","Status","untappd"]])
# pd.reset_option('display.width')
pd.reset_option('display.max_rows')

Total need to research 49


,Name,License Date,Brewery License,Status,untappd
7,Allen's Landing Brewing,1/16/2015,Brewery,Open,[{'no_results': True}]
13,BCS-Zoigl,12/1/2016,Brewery,Open,[{'no_results': True}]
19,Braman Brewery,5/4/2015,Brewery,Open,"[{'brewery': {'brewery_id': 48985, 'beer_count': 1, 'brewery_name': 'Bratman Brewery (Homebrew)', 'brewery_slug': 'bratman-brewery', 'brewery_page_url': '/w/bratman-brewery/48985', 'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png', 'country_name': 'United States', 'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}]"
24,City Orchard,6/18/2018,Brewery,Open,"[{'brewery': {'brewery_id': 364659, 'beer_count': 1, 'brewery_name': 'Orchard City Brewing (Homebrew)', 'brewery_slug': 'orchard-city-brewing', 'brewery_page_url': '/w/orchard-city-brewing/364659', 'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png', 'country_name': 'United States', 'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}]"
25,Comfort Brewing,8/29/2016,Brewery,Open,"[{'brewery': {'brewery_id': 209165, 'beer_count': 24, 'brewery_name': 'Cold Comfort Brewing (Homebrew)', 'brewery_slug': 'cold-comfort-brewing', 'brewery_page_url': '/ColdComfortBrewing', 'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png', 'country_name': 'United States', 'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}, {'brewery': {'brewery_id': 124177, 'beer_count': 7, 'brewery_name': 'Comfort Brewing Co. (Homebrew)', 'brewery_slug': 'comfort-brewing-co', 'brewery_page_url': '/w/comfort-brewing-co/124177', 'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png', 'country_name': 'United States', 'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}, {'brewery': {'brewery_id': 321023, 'beer_count': 3, 'brewery_name': 'Cold Comfort Brewing (Homebrew)', 'brewery_slug': 'cold-comfort-brewing', 'brewery_page_url': '/w/cold-comfort-brewing/321023',..."
30,Deep Ellum FTW,9/5/2018,Brewery,Open,[{'no_results': True}]
31,Dej Buh Stesti Brewing,1/8/2016,Brewery,Open,[{'no_results': True}]
39,Friends And Allies Brewing,5/26/2016,Brewery,Open,[{'no_results': True}]
55,Lake Austin Ales (Operating in Celis),5/4/2015,Brewery,Open,[{'no_results': True}]
61,Long Wood Spoon Brewing,6/23/2015,Brewery,Open,[{'no_results': True}]


Removed names to help with search and start over

In [134]:
# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)
  

names_to_be_removed = [
    "Inc"
#     "Arlington",
#     "/ Austin Java",
#     "Northwest Highway",
#     "Co-Op",
#     " Co",
#     "Llc"
]
for brewery in all_breweries:
#     any(x in str for x in a)

    if any(x in brewery["Name"] for x in names_to_be_removed):
#         
        for w in names_to_be_removed:
            brewery["Name"] = brewery["Name"].replace(w,"")
        
        del brewery["untappd"]
        print(brewery["Name"])
        

    
# with open("../data/all_breweries.pkl", "wb") as fp:
#     pickle.dump(all_breweries, fp)

Spoetzl Brewery 
Brick Oven Pizza Of Abilene 
Fredericksburg Brewing  
Padre Island Brewing 
San Gabriel River Brewery 
Walking Stick Brewing  


Check for Duplicates and do something with them

In [147]:
import pickle

with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)
    
#remove duplicates
brewery_count = pd.DataFrame(all_breweries).groupby(["Name","brewery_city"])["Status"].count()
dups = brewery_count.loc[ brewery_count > 1].reset_index()
dups = dups["Name"].to_list()
print("Duplicates to be removed", dups)
all_breweries = [ brewery for brewery in all_breweries if brewery["Name"] not in dups ]

with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)

Duplicates to be removed ['Flix Brewhouse', 'Freetail Brewing Co', 'Hops And Grain', 'Malai Kitchen', 'Pinthouse Pizza']


Get the Zip Code

In [161]:
import pickle
from uszipcode import SearchEngine

with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

for brewery in all_breweries:
    if "brewery_id" in brewery and abs(brewery["brewery_lat"]) > 0 and abs(brewery["brewery_lng"]) > 0:
        search = SearchEngine(simple_zipcode=False)
        result = search.by_coordinates(brewery["brewery_lat"],brewery["brewery_lng"], radius=30, returns=6)
        first_zip = result[0].to_dict()
        brewery["brewery_zipcode"] = first_zip["zipcode"]
        neighboring_zips = [zip.to_dict()["zipcode"] for zip in result[1:3]]
        brewery["neighboring_zips"] = neighboring_zips

print( all_breweries[1] )

with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)

{'Name': '2 Docs Brewing Co', 'Street Address': 502, 'Road': 'TEXAS AVENUE', 'City': 'LUBBOCK', 'State': 'TX', 'Brewery License': 'Brewery', 'License Date': '10/5/2018', 'Status': 'Open', 'untappd': [{'brewery': {'brewery_id': 420745, 'beer_count': 22, 'brewery_name': 'Two Docs Brewing Co.', 'brewery_slug': 'two-docs-brewing-co', 'brewery_page_url': '/TwoDocsBrewingCo', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Lubbock', 'brewery_state': 'TX', 'lat': 33.5909, 'lng': -101.846}}}], 'brewery_id': 420745, 'brewery_info': {'brewery': {'brewery_id': 420745, 'brewery_name': 'Two Docs Brewing Co.', 'brewery_slug': 'two-docs-brewing', 'brewery_page_url': '/TwoDocsBrewingCo', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg', 'brewery_label_hd': '', 'country_name': 'United States', 'brewery_in_production': 0, 'is_independent': 1, 'claimed

In [169]:
columns = [
    "brewery_name",
    "Brewery License",
    "License Date",
    "Status",
    "brewery_in_production",
    "brewery_id",
    "brewery_address",
    "brewery_city",
    "brewery_state",
    "brewery_lat",
    "brewery_lng",
    "beer_count",
    "rating_count",
    "rating_score",
    "age_on_service",
    "brewery_zipcode",
    "neighboring_zips",
    "checkins_total_count",
    "checkins_unique_count",
    "checkins_monthly_count",
    "checkins_weekly_count"
]

breweries = pd.DataFrame(all_breweries)



{'Name': '2 Docs Brewing Co', 'Street Address': 502, 'Road': 'TEXAS AVENUE', 'City': 'LUBBOCK', 'State': 'TX', 'Brewery License': 'Brewery', 'License Date': '10/5/2018', 'Status': 'Open', 'untappd': [{'brewery': {'brewery_id': 420745, 'beer_count': 22, 'brewery_name': 'Two Docs Brewing Co.', 'brewery_slug': 'two-docs-brewing-co', 'brewery_page_url': '/TwoDocsBrewingCo', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Lubbock', 'brewery_state': 'TX', 'lat': 33.5909, 'lng': -101.846}}}], 'brewery_id': 420745, 'brewery_info': {'brewery': {'brewery_id': 420745, 'brewery_name': 'Two Docs Brewing Co.', 'brewery_slug': 'two-docs-brewing', 'brewery_page_url': '/TwoDocsBrewingCo', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg', 'brewery_label_hd': '', 'country_name': 'United States', 'brewery_in_production': 0, 'is_independent': 1, 'claimed